In [41]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph, MessagesState



# Initialize the OpenAI model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [42]:
# source: https://langchain-ai.github.io/langgraph/#step-by-step-breakdown
# Function to call the model
def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {"messages": [response]}

In [43]:
# Define the workflow
# we initialize graph (StateGraph) by passing state schema (in our case MessagesState)
# MessagesState is a prebuilt state schema that has one attribute -- 
# a list of LangChain Message objects, as well as logic for merging the updates from each node into the state
workflow = StateGraph(MessagesState)

In [44]:
workflow.add_node("agent", call_model)

In [45]:
workflow.set_entry_point("agent")

In [46]:
app = workflow.compile()

In [47]:
response = app.invoke({"messages": [HumanMessage(content="Tell me a joke")]})

In [48]:
response['messages'][-1].content

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'